# Aligning data — example notebook

In [ ]:
from imports import * # contains necessary packages
from load_data_functions import * # contains functions for reading in and aligning data

---
## Reading in data as arrays of SunPy maps
IRIS 1400, AIA 171 and 1600, and Solar Orbiter/EUI 174 data

In [ ]:
# Reading in AIA 171 data -- USING DATA DOWNLOADED FROM JSOC
folder = 'ex_data/JSOC_20221206_1621' # path to folder with AIA data
aia171_maps = get_aia_data(folder) # this reads in the data contained in "folder" and creates SunPy maps

In [ ]:
# Reading in AIA 1600 data -- USING DATA DOWNLOADED FROM JSOC
folder = 'ex_data/JSOC_20221210_179'
aia1600_maps = get_aia_data(folder)

In [ ]:
# Reading in IRIS 1400 data -- USING DATA DOWNLOADED FROM IRIS WEBSITE
filename = 'ex_data/iris_l2_20220305_150930_3600607418_SJI_1400_t000.fits' # path to fits with IRIS data
iris1400_maps = get_iris_data(filename) # this reads in the data contained in "filename" and creates SunPy maps

In [ ]:
# READING IN SOLAR ORBITER/EUI 174 DATA -- GETTING DATA USING SOAR
# (this only works for me when I'm connected LMSAL internet)
eui_maps = get_eui_data(start_date='2022-03-05T15:50:00', 
                        end_date='2022-03-06T00:00:01')

---
## Cut observations so they cover the same time

In [ ]:
# get overlap times between EUI and IRIS observations
start_time, end_time = obs_overlap_times(eui_maps, iris1400_maps)
print(start_time, end_time)

In [ ]:
aia171_maps_cut = np.array(cut_obs(aia171_maps, start_time, end_time))
aia1600_maps_cut = np.array(cut_obs(aia1600_maps, start_time, end_time))
iris1400_maps_cut = np.array(cut_obs(iris1400_maps, start_time, end_time))

# ensure EUI is sorted by time -- usually not if loaded in using SOAR
eui_sorted, sorted_i = sort_data(eui_maps)
eui_maps_cut = np.array(cut_obs(eui_sorted, start_time, end_time))

---
## Matching observations 

AIA, IRIS, and EUI observe at difference cadences. To account for this, "match_obs" matches each image of the EUI observation with an image from the IRIS/AIA observation that is the closest in time.

This makes both sets of observations contain the same number of images.

(If these the instruments were at different distances from Earth, "match_obs" would account for that and match images with respective to Earth time.)

In [ ]:
# MATCHING OBSERVATIONS TO EUI -- MAKE ALL SAME LENGTH

times_to_match = get_earth_times(eui_maps_cut) # get earth times

eui_matched = eui_maps_cut
iris1400_matched = match_obs(iris1400_maps_cut, times_to_match)
aia171_matched = match_obs(aia171_maps_cut, times_to_match)
aia1600_matched = match_obs(aia1600_maps_cut, times_to_match)

---

## Correct for solar rotation
The Sun rotates at a rate of roughly 2 km/s. We need to correct for this movement or else we'll see our images drift over time!

In [ ]:
# already corrected for in IRIS -- rotation already corrected for in data downloaded from IRIS website

aia171_rot = correct_diff_rot_obs(aia171_matched, ref_i=0) # align to first observation in the array
aia1600_rot = correct_diff_rot_obs(aia1600_matched, ref_i=0)
eui_rot = correct_diff_rot_obs(eui_matched, ref_i=0)

---

## Align all observations to the same reference frame
AIA, IRIS, and EUI are instruments on different satellites so observe the Sun from different perspectives. To account for this, we'll align all images to the same reference image — the middle AIA observation.

In [ ]:
# ALIGN ALL OBS TO THE SAME REFERENCE FRAME
ref_obs = iris1400_matched[len(iris1400_matched)//2] # middle IRIS observation

In [ ]:
aia171_aligned = align_obs_to_reference(aia171_rot, ref_obs)

In [ ]:
aia1600_aligned = align_obs_to_reference(aia1600_rot, ref_obs)

In [ ]:
iris1400_aligned = align_obs_to_reference(iris1400_matched, ref_obs)

In [ ]:
eui_aligned = align_obs_to_reference(eui_rot, ref_obs)

---
## Manual Alignment
In theory, these IRIS, AIA, and EUI observations should be well aligned. Unfortunately, there are usually some manual alignment that that needs to be done too.

### Aligning Solar Orbiter Obervation with Itself
In some Solar Orbiter observations, the instrument pointing is not completely stable, so there is some jitter (like in this example observation). To correct for this, we'll cross correlate a subregion that is relatively stable over time. Different methods of cross correlating and different regions used for cross correlation may work better for different observations, so it's worth trying different things!

In [ ]:
# subregion used for cross correlation -- moss region
xlims = [20, 200]
ylims = [0, 140]

eui_aligned_zoomed = sunpy.map.Map(list(create_obs_submaps(eui_aligned, xlims, ylims)),
                                   sequence=True)

In [ ]:
eui_aligned_zoomed[0].plot()

In [ ]:
# cross correlating one EUI image with the next image
# (image 1 cross correlated with image 2, image 2 with image 3, ...)

whole_obs1 = eui_aligned_zoomed[:-1]
whole_obs2 = eui_aligned_zoomed[1:]

img1_list = [obs.data for obs in whole_obs1]
img2_list = [obs.data for obs in whole_obs2]

shifts = p_map(cross_correlate, img1_list, img2_list)

In [ ]:
# sum the shift values we calculate in the image before so all image are aligned with the first one
# (the shift we apply for image 3 is the sum of the shifts we calculated for image 2 and image 3)

y = []; x = []
for i in shifts:
    y.append(i[0]); x.append(i[1])

# sum the shifts that come before
x_summed = [np.sum(x[:i]) for i in range(len(x))]
y_summed = [np.sum(y[:i]) for i in range(len(y))]

shifts_summed = []
for i in range(len(y_summed)):
        shifts_summed.append(np.array([y_summed[i], x_summed[i]]))

In [ ]:
eui_shifted = shift_obs(eui_aligned, shifts_summed) # applying the shifts to the EUI images

### Checking EUI and AIA Alignment
Now, that EUI is aligned with itself, let's check the alignment with AIA. While we aligned these observations using the header information given in the fits files, sometimes it's not the most accurate. It's always worth double checking the alignment! For the most part, observations from the same instrument should be well aligned with eachother. 

Since EUI is aligned with itself and AIA is well aligned with itself, we will calculate and apply the mean shift value. Meaning the each EUI image will be shifted by the same amount.

In [ ]:
img1_list = [obs.data for obs in eui_shifted]
img2_list = [obs.data for obs in aia171_aligned]

shifts = p_map(cross_correlate, img2_list, img1_list) # performing cross correlation

In [ ]:
y = []; x = []
for i in shifts:
    y.append(i[0]); x.append(i[1])

# 3 sigma cutting -- omitting outliers
# x_std = np.nanmean(x)
# x_mean = np.nanmean(x)
# x_cut = np.array(x)[np.where(x < x_mean+3*x_std)]

# y_std = np.nanstd(y)
# y_mean = np.nanmean(y)
# y_cut = np.array(y)[np.where(y < y_mean+3*y_std)]


# plotting
%matplotlib inline
fig, ax = plt.subplots(1, 2, figsize=(12, 4))

ax[0].set_title('x')
ax[0].plot(x, c='k')

ax[1].set_title('y')
ax[1].plot(y, c='k')


plt.show()

print('X mean shift: ', np.nanmean(x))
print('Y mean shift: ', np.nanmean(y))

In [ ]:
# create constant shift array of mean X and Y shift values
shifts_cst = [np.array([np.nanmean(y), np.nanmean(x)]) for i in range(len(shifts))]

In [ ]:
eui_shifted_again = shift_obs(eui_shifted, shifts_cst) # shifting EUI to match AIA

### Checking AIA and IRIS alignment

Now, we've shifted the EUI 174 images so they're aligned with AIA 171. This process should be repeated for ensuring AIA and IRIS data are well aligned. To do this, it's best to compare data from similar wavelengths because they'll share similar features. We'll use AIA's 1600 channel and IRIS's 1400 channel.

In [ ]:
img1_list = [obs.data for obs in iris1400_aligned]
img2_list = [obs.data for obs in aia1600_aligned]

shifts = p_map(cross_correlate_mask, img2_list, img1_list)

In [ ]:
y = []; x = []
for i in shifts:
    y.append(i[0]); x.append(i[1])

# 3 sigma cutting
x_std = np.nanstd(x)
x_mean = np.nanmean(x)
x_cut = np.array(x)[np.where((x < x_mean+3*x_std) & (x > x_mean-x_std))]

y_std = np.nanstd(y)
y_mean = np.nanmean(y)
y_cut = np.array(y)[np.where((y < y_mean+3*x_std) & (y > y_mean-y_std))]


# plotting
%matplotlib inline
fig, ax = plt.subplots(1, 2, figsize=(12, 4))

ax[0].set_title('x')
ax[0].plot(x_cut, c='k')

ax[1].set_title('y')
ax[1].plot(y_cut, c='k')

plt.show()

print('X mean shift: ', np.nanmean(x_cut))
print('Y mean shift: ', np.nanmean(y_cut))

In [ ]:
# create constant shift array
shifts_cst = [np.array([np.nanmean(y_cut), np.nanmean(x_cut)]) for i in range(len(shifts))]

In [ ]:
iris1400_shifted = p_map(apply_shift, iris1400_aligned, shifts_cst) # shifting IRIS to match AIA

---
## Saving the aligned data

You can save data to files in a variety of formats, here we'll save out aligned and shifted data as pickle files. This format is well suited for python. 

In [ ]:
save_as_pickle(aia171_aligned, 'aia171_data.pickle')
save_as_pickle(aia1600_aligned, 'aia1600_data.pickle')
save_as_pickle(iris1400_shifted, 'iris1400_data.pickle')
save_as_pickle(eui_shifted_again, 'eui174_data.pickle')

save_as_pickle(aia171_matched, 'aia171_matched.pickle') # contains time information
save_as_pickle(aia1600_matched, 'aia1600_matched.pickle')
save_as_pickle(iris1400_matched, 'iris1400_matched.pickle')
save_as_pickle(eui_matched, 'eui174_matched.pickle')

### Reading in pickle file

In [ ]:
aia171_data = read_from_pickle('aia171_data.pickle')
aia1600_data = read_from_pickle('aia1600_data.pickle')
iris1400_data = read_from_pickle('iris1400_data.pickle')
eui_data = read_from_pickle('eui174_data.pickle')

aia171_matched = read_from_pickle('aia171_matched.pickle')
aia1600_matched = read_from_pickle('aia1600_matched.pickle')
iris1400_matched = read_from_pickle('iris1400_matched.pickle')
eui_matched = read_from_pickle('eui174_matched.pickle')

---
## Plotting the aligned data

In [ ]:
# PLOT 4 DATASETS
i = 100

map1 = iris1400_data[i] # iris
map1_time = iris1400_matched[i].date # need to access the non-aligned metadata for the correct time
map1_name = 'IRIS 1400'
map1_cmap = 'irissji1400'
map1_norm = plt.Normalize(-5, 15)

map2 = aia171_data[i] # aia 171
map2_time = aia171_matched[i].date
map2_name = 'AIA 171'
map2_cmap = 'sdoaia171'
map2_norm = plt.Normalize(0, 2700)

map3 = aia1600_data[i] # aia 1600
map3_time = aia1600_matched[i].date
map3_name = 'AIA 1600'
map3_cmap = 'sdoaia1600'
map3_norm = plt.Normalize(0, 180)

map4 = eui_data[i] # eui shifted to aia 171
map4_time = eui_matched[i].date
map4_name = 'EUI 174'
map4_cmap = 'solar orbiterhri_euv174'
map4_norm = plt.Normalize(10, 3000)


####

%matplotlib notebook
plt.rcParams['figure.figsize'] = [10, 10]
fig, axs = plt.subplots(2, 2)

axs[0,0].axis('off'); axs[0,1].axis('off'); axs[1,0].axis('off'); axs[1,1].axis('off')

# map 1 -- IRIS 1400
ax1 = fig.add_subplot(221, projection=map1.wcs)
ax1.imshow(map1.data, cmap=map1_cmap, norm=map1_norm)
x, y = map1.data.shape
ax1.set_title(f'{map1_name}\n{map1_time.datetime.time()}', fontsize=8)

# map 2 -- AIA 171
ax2 = fig.add_subplot(222, projection=map1.wcs, sharex=ax1, sharey=ax1)
ax2.imshow(map2.data, cmap=map2_cmap, norm=map2_norm)
x, y = map2.data.shape
ax2.set_title(f'{map2_name}\n{map2_time.datetime.time()}', fontsize=8)

# map 3 -- AIA 1600
ax3 = fig.add_subplot(223, projection=map1.wcs, sharex=ax1, sharey=ax1)
ax3.imshow(map3.data, cmap=map3_cmap, norm=map3_norm)
x, y = map3.data.shape
ax3.set_title(f'{map3_name}\n{map3_time.datetime.time()}', fontsize=8)

# map 4 -- EUI 174
ax4 = fig.add_subplot(224, projection=map1.wcs, sharex=ax1, sharey=ax1)
ax4.imshow(map4.data, cmap=map4_cmap, norm=map4_norm)
x, y = map4.data.shape
ax4.set_title(f'{map4_name}\n{map4_time.datetime.time()}', fontsize=8)

plt.tight_layout(pad = 4)
plt.show()